In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [28]:
unsw = pd.read_csv("../datasets/UNSW_NB15_training-set.csv")

# Load data
"""Load unsw dataset"""
y = unsw.iloc[:,-1:]
#one-hot-encode parameters
proto = pd.get_dummies(unsw['proto'])
state = pd.get_dummies(unsw['state'])
service = pd.get_dummies(unsw['service'])
service = service.iloc[:,1:]
raw_data = unsw.values

#remove encoded parameters and add one hot
for x in ['proto', 'state', 'service','label','attack_cat']:
    unsw = unsw.drop(x,axis = 1)
    
for x in [proto, state, service]:
    unsw = unsw.join(x)
    
X = unsw.iloc[:,:-2]

X1 = X[:20000] 
y1 = y[:20000]

X2 = X[20000:40000]
y2 = y[20000:40000]

X_test = X[40000:]
y_test = y[40000:]

In [31]:
X_train = torch.FloatTensor(X1.to_numpy().ravel())
X_test = torch.FloatTensor(X_test.to_numpy().ravel())
y_train = torch.LongTensor(y1.to_numpy().ravel())
y_test = torch.LongTensor(y_test.to_numpy().ravel())

In [32]:
y_train

tensor([0, 0, 0,  ..., 1, 1, 1])

In [39]:
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

In [40]:
input_dim  = 188
output_dim = 1
model = NeuralNetworkClassificationModel(input_dim,output_dim)

In [41]:
# creating our optimizer and loss function object
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [42]:
def train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(X_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)
        


        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        
        output_test = model(X_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

In [43]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [44]:
train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3760000 and 188x128)

In [17]:
y_train

tensor([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]])